<a href="https://colab.research.google.com/github/daochenzha/douzero-colab/blob/main/douzero-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook for DouZero

First, install the dependencies with pip

In [1]:
%pip install torch==1.6.0
%pip install GitPython==3.0.5
%pip install gitdb2==2.0.6
%pip install rlcard

     |████████████████████████████████| 748.8MB 25kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.6.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.6.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
     |████████████████████████████████| 460kB 7.0MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
     |████████████████████████████████| 71kB 6.4MB/s 
  Found existing installation: gitdb2 4.0.2
    Uninstalling gitdb2-4.0.2:
      Successfully uninstalled gitdb2-4.0.2
     |████████████████████████████████| 256kB 7.5MB/s 
  Created wheel for rlcard: filename=rlcard-1.0.2-cp37-none-any.whl size=307202 sha256=b4c4cb8f6f7e616c8d91b373b636be6c42d4bfa8939273af2b6616df247efa4e
  Stored in directory: /root/.cache/pip/wheels/77/89/ca/f06d1e3aa24faf0cfd39984608f57e659a7167682a9db5a70

Then install douzero with pip

In [1]:
%pip install douzero

Train douzero with

In [5]:
import os

from douzero.dmc import parser, train

if __name__ == '__main__':
    # To get all defaults:
    flags = parser.parse_args([])
    train(flags)

Creating log directory: douzero_checkpoints/douzero
Saving arguments to douzero_checkpoints/douzero/meta.json
Saving messages to douzero_checkpoints/douzero/out.log
Saving logs data to douzero_checkpoints/douzero/logs.csv
Saving logs' fields to douzero_checkpoints/douzero/fields.csv
[INFO:61 dmc:173 2021-06-17 14:12:21,064] Saving checkpoint to douzero_checkpoints/douzero/model.tar
[INFO:61 dmc:215 2021-06-17 14:12:21,432] After 0 (L:0 U:0 D:0) frames: @ 0.0 fps (L:0.0 U:0.0 D:0.0) Stats:
{'loss_landlord': 0,
 'loss_landlord_down': 0,
 'loss_landlord_up': 0,
 'mean_episode_return_landlord': 0,
 'mean_episode_return_landlord_down': 0,
 'mean_episode_return_landlord_up': 0}
[INFO:61 dmc:215 2021-06-17 14:12:26,451] After 0 (L:0 U:0 D:0) frames: @ 0.0 fps (L:0.0 U:0.0 D:0.0) Stats:
{'loss_landlord': 0,
 'loss_landlord_down': 0,
 'loss_landlord_up': 0,
 'mean_episode_return_landlord': 0,
 'mean_episode_return_landlord_down': 0,
 'mean_episode_return_landlord_up': 0}
[INFO:61 dmc:215 2021-0

For evaluation, for mount your 
Google drive


In [2]:
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/gdrive')

Mounted at /content/gdrive


You can download the "baselines" from the link in the repo and put it in your own Google Drive. We can have a double-check with `ls`:

In [3]:
!ls /content/gdrive/MyDrive/baselines

douzero_ADP  douzero_WP  sl


Then we generate some evaluating data and save it in your drive:

In [4]:
num_games = 1000
output_pickle = '/content/gdrive/MyDrive/douzero_eval_data.pkl'

import pickle
import numpy as np

deck = []
for i in range(3, 15):
    deck.extend([i for _ in range(4)])
deck.extend([17 for _ in range(4)])
deck.extend([20, 30])

def generate():
    _deck = deck.copy()
    np.random.shuffle(_deck)
    card_play_data = {'landlord': _deck[:20],
                      'landlord_up': _deck[20:37],
                      'landlord_down': _deck[37:54],
                      'three_landlord_cards': _deck[17:20],
                      }
    for key in card_play_data:
        card_play_data[key].sort()
    return card_play_data
  


print("output_pickle:", output_pickle)
print("generating data...")

data = []
for _ in range(num_games):
    data.append(generate())

print("saving pickle file...")
with open(output_pickle,'wb') as g:
    pickle.dump(data,g,pickle.HIGHEST_PROTOCOL)

output_pickle: /content/gdrive/MyDrive/douzero_eval_data.pkl
generating data...
saving pickle file...


Then, we can evaluate pre-trained ADP model with RLCard baseline with

In [5]:
from douzero.evaluation.simulation import evaluate

evaluate(landlord='/content/gdrive/MyDrive/baselines/douzero_ADP/landlord.ckpt',
         landlord_up='rlcard',
         landlord_down='rlcard',
         eval_data=output_pickle,
         num_workers=5)

WP results:
landlord : Farmers - 0.853 : 0.147
ADP results:
landlord : Farmers - 2.466 : -2.466
